In [1]:
! pip install mne==0.19.2
! pip install torch
! pip install skorch

In [2]:
%cd /content/drive/MyDrive/
# raw_data is imported from global config
root_dir='/content/drive/MyDrive/'  # ChangeThis
result_dir=root_dir+'grasp/TSception/result_subject2/'

/content/drive/MyDrive


In [3]:
import os, re
location=os.getcwd()
print(location)

/content/drive/MyDrive


In [4]:
print(re.compile('/content/drive').match(location))

<re.Match object; span=(0, 14), match='/content/drive'>


In [5]:
import sys, importlib
#importlib.reload(sys.modules['grasp.config'])
from grasp.config import *
# orveride the data_dir in config file
#data_dir='/content/drive/MyDrive/data/' # googleDrive

In [6]:
data_dir

'/content/drive/MyDrive/data/'

In [7]:
import torch
from skorch.helper import predefined_split
from torch import nn
import torch.nn.functional as F
from skorch import NeuralNetRegressor
from skorch.callbacks import Callback
import numpy as np
import matplotlib.pyplot as plt

from grasp.TSception.Models import TSception2
from grasp.utils import freq_input,SEEGDataset,set_random_seeds,cuda_or_cup,windTo3D_x, windTo3D_y
from grasp.config import root_dir, tmp_dir
from grasp.myskorch import plotPrediction, MyRegressor


In [8]:
# reload function
import sys, importlib
importlib.reload(sys.modules['grasp.TSception.Models'])
importlib.reload(sys.modules['grasp.utils'])
importlib.reload(sys.modules['grasp.config'])
from grasp.TSception.Models import TSception,TSception2
from grasp.utils import freq_input,SEEGDataset

In [9]:
# import from another folder
import sys
sys.path.insert(1, '/content/drive/MyDrive/examples')
from IMV_LSTM.networks import IMVTensorLSTM

In [10]:
import inspect as i
import sys
#sys.stdout.write(i.getsource(rawData2));

In [11]:
device=cuda_or_cup()
seed = 123456789  # random seed to make results reproducible
# Set random seed to be able to reproduce results
set_random_seeds(seed=seed)


GPU computing:   False


In [12]:
%%capture
# suppress the output
sid=6
sampling_rate=1000
traindata, valdata, testdata = freq_input(sid,split=True,move2=True)  # (chns, 15000/15001, 118) (channels, time, trials)
##traindata, valdata, testdata = rawData2('raw','all',move2=True)
traindata = traindata.transpose(2, 0, 1)  # (118, 20, 15000) (trials,channels,  time)
valdata = valdata.transpose(2, 0, 1) # (8, 20, 15000)
testdata = testdata.transpose(2, 0, 1)  # (8, 20, 15000)

trainx, trainy = traindata[:, :-2, :], traindata[:, -2, :] #-2 is real force, -1 is target
valx, valy = valdata[:, :-2, :], valdata[:, -2, :]
testx, testy = testdata[:, :-2, :], testdata[:, -2, :]
# trainx.shape(trials, channels,times)
# trainy.shape(trials, times)

In [13]:
# put train and valdate togather, CV will do the split automatically
train_and_valx=np.concatenate((trainx,valx),axis=0)
train_and_valy=np.concatenate((trainy,valy),axis=0)
del trainx, valx, trainy, valy,testx,testy, traindata, valdata,testdata

In [14]:
# prevent crash on memory usage
train_and_valx=train_and_valx[:60,:,:]
train_and_valy=train_and_valy[:60,:]

In [15]:
#samples=trainx.shape[0]
chnNum=train_and_valx.shape[1]
step=50 #ms
T=1000 #ms
totalLen=train_and_valx.shape[2] #ms
batch_size=int((totalLen-T)/step) # 280


In [ ]:
train_and_val_wind_x=[]
for trial in range(train_and_valx.shape[0]):
    print(str(trial)+' out of '+str(train_and_valx.shape[0]))
    train_and_val_wind_x.append([])
    train_and_val_wind_x[trial]=windTo3D_x(train_and_valx[trial],T,step)
del train_and_valx
train_and_val_final_x=np.stack( train_and_val_wind_x) # (10, 280, 1, 90, 1000)
del train_and_val_wind_x

In [19]:
train_and_val_wind_y=[]
for trial in range(train_and_valy.shape[0]):
    train_and_val_wind_y.append([])
    train_and_val_wind_y[trial]=windTo3D_y(train_and_valy[trial],T,step)
del train_and_valy
train_and_val_final_y=np.stack( train_and_val_wind_y) # (10, 280, 1)
del train_and_val_wind_y

In [20]:
learning_rate=0.001
num_T = 3 # (6 conv2d layers) * ( 3 kernel each layer)
num_S = 3
num_ST=3
dropout=0.5
#Lambda = 1e-6
Lambda = 0
# do not initialize here. Do it in net definition.
#model=TSception2(T, step, sampling_rate,chnNum, num_T, num_S,batch_size,dropout).float()

#if(enable_cuda):
#	model.cuda()

In [21]:
from skorch.callbacks import Checkpoint, TrainEndCheckpoint, EarlyStopping
cp = Checkpoint(dirname=result_dir) # Will automatically save the model when there is an improvement during validation.
train_end_cp = TrainEndCheckpoint(dirname=result_dir) # save model and the end of the training

early_stopping_callbacks = EarlyStopping(monitor='valid_loss', lower_is_better=True, patience=5)

In [23]:
net = MyRegressor(
    module=TSception2,
    module__wind_size=T,
    module__step=step,
    module__sampling_rate=sampling_rate,
    module__chnNum=chnNum,
    module__num_T=num_ST,
    module__num_S=num_ST,
    module__batch_size=batch_size,
    module__dropout=dropout,
    lambda1=Lambda,
    iterator_train__shuffle=True,
    #train_split=predefined_split(val_ds), #disable this when do grid search
    train_split=None, # grid search
    verbose=0, # grid search
    max_epochs=100,
    lr=learning_rate,
    batch_size=1,
    optimizer=torch.optim.Adam,
    criterion = nn.MSELoss,
    callbacks=[('plotPrediction', plotPrediction(result_dir)),cp,train_end_cp,early_stopping_callbacks], # early_stopping_callbacks
    device = device
)


In [24]:
from sklearn.model_selection import GridSearchCV
params = {
    'lr': [0.001, 0.003],
    'module__num_S':[2,3],
    'module__num_T':[2,3,4,5,6],
    'module__dropout':[0.1,0.3,0.5,0.7],
    'optimizer': [torch.optim.Adam, torch.optim.Adagrad ,torch.optim.SGD],
    'optimizer__weight_decay':[0, 1e-04, 1e-03],
    #'max_epochs': [100,200, 300],
    #'module__num_units': [10, 20],
}
'''
params = {
    'lr': [0.001, 0.003,0.005,0.01],
    'module__num_S':[2,3,4,5,6],
    'module__num_T':[2,3,4,5,6],
    'module__dropout':[0.1,0.3,0.5,0.7],
    'optimizer': [torch.optim.Adam, torch.optim.Adagrad ,torch.optim.SGD],
    'optimizer__weight_decay':[0, 1e-04, 1e-03],
    #'max_epochs': [100,200, 300],
    #'module__num_units': [10, 20],
}
'''
# note
# 1, gridsearch will do the training and validate set split itself. so set train_split=False according to https://nbviewer.jupyter.org/github/skorch-dev/skorch/blob/master/notebooks/Basic_Usage.ipynb#Performing-a-grid-search
# 2, how to search on different optimizor?

"\nparams = {\n    'lr': [0.001, 0.003,0.005,0.01],\n    'module__num_S':[2,3,4,5,6],\n    'module__num_T':[2,3,4,5,6],\n    'module__dropout':[0.1,0.3,0.5,0.7],\n    'optimizer': [torch.optim.Adam, torch.optim.Adagrad ,torch.optim.SGD],\n    'optimizer__weight_decay':[0, 1e-04, 1e-03],\n    #'max_epochs': [100,200, 300],\n    #'module__num_units': [10, 20],\n}\n"

In [25]:
#net.fit(train_ds,y=None)
gs = GridSearchCV(net, params, refit=False, cv=5, scoring='neg_mean_squared_error', verbose=2) # score: higher is better
gs.fit(train_and_val_final_x,train_and_val_final_y)


Fitting 5 folds for each of 720 candidates, totalling 3600 fits
[CV] lr=0.001, module__dropout=0.1, module__num_S=2, module__num_T=2, optimizer=<class 'torch.optim.adam.Adam'>, optimizer__weight_decay=0 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Epoch 1.


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
RuntimeError: expected scalar type Double but found Float

  FitFailedWarning)
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   14.8s remaining:    0.0s


[CV]  lr=0.001, module__dropout=0.1, module__num_S=2, module__num_T=2, optimizer=<class 'torch.optim.adam.Adam'>, optimizer__weight_decay=0, total=  14.6s
[CV] lr=0.001, module__dropout=0.1, module__num_S=2, module__num_T=2, optimizer=<class 'torch.optim.adam.Adam'>, optimizer__weight_decay=0 
Epoch 1.
[CV]  lr=0.001, module__dropout=0.1, module__num_S=2, module__num_T=2, optimizer=<class 'torch.optim.adam.Adam'>, optimizer__weight_decay=0, total=   3.7s
[CV] lr=0.001, module__dropout=0.1, module__num_S=2, module__num_T=2, optimizer=<class 'torch.optim.adam.Adam'>, optimizer__weight_decay=0 
Epoch 1.
[CV]  lr=0.001, module__dropout=0.1, module__num_S=2, module__num_T=2, optimizer=<class 'torch.optim.adam.Adam'>, optimizer__weight_decay=0, total=   3.7s
[CV] lr=0.001, module__dropout=0.1, module__num_S=2, module__num_T=2, optimizer=<class 'torch.optim.adam.Adam'>, optimizer__weight_decay=0 
Epoch 1.
[CV]  lr=0.001, module__dropout=0.1, module__num_S=2, module__num_T=2, optimizer=<class 

KeyboardInterrupt: ignored

In [ ]:
import sklearn
#sorted(sklearn.metrics.SCORERS.keys())

In [ ]:
print(gs.best_score_, gs.best_params_)